In [3]:
import pandas as pd
import os
import numpy as np

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

In [89]:
np.random.seed(0)
#Read file into pandas dataframe 
nanDict= {}
df = pd.read_excel('default of credit card clients.xls', header=1, skiprows=0, index_col=0, na_values=nanDict)
df.rename(index=str, columns={'default paument next month': 'defaultPaymentNextMonth'}, inplace=True)

#Drop the rows including data where parameters are out of range
df=df.drop(df[df.SEX<1].index)
df=df.drop(df[df.SEX<2].index)
df=df.drop(df[(df.EDUCATION <1)].index)
df=df.drop(df[(df.EDUCATION >4)].index)
df=df.drop(df[df.MARRIAGE<1].index)
df=df.drop(df[df.MARRIAGE>3].index)


#Features and targets
X= df.loc[:, df.columns != 'defaultPaymentNextMonth'] 
y= df.loc[:, df.columns == 'defaultPaymentNextMonth']


#test= np.where(X[:,1]<1, X[:,1], X[:,1] >2)[0]
#test
X= df.loc[:, df.columns != 'defaultPaymentNextMonth'].values
y= df.loc[:, df.columns == 'defaultPaymentNextMonth'].values


outlier_gender1 = np.where(X[:,1] < 1)[0]
outlier_gender2 = np.where(X[:,1] > 2)[0]

outlier_education1 = np.where(X[:,2] < 1)[0]
outlier_education2 = np.where(X[:,2] > 4)[0]

outlier_marital1 = np.where(X[:,3] < 1)[0]
outlier_marital2 = np.where(X[:,3] > 3)[0]

inds = np.concatenate((outlier_gender1,
                        outlier_gender2,
                        outlier_education1,
                        outlier_education2,
                        outlier_marital1,
                        outlier_marital2))

outlier_rows = np.unique(inds)
X = np.delete(X, outlier_rows, axis=0)
y = np.delete(y, outlier_rows, axis=0)

y

In [92]:
# Categorical variables to one-hot's
onehotencoder = OneHotEncoder(categories="auto")

#OneHot encoder for column 1,2,3 [sex,education,marriage]
#Designmatrix
X = ColumnTransformer(
    [('onehotencoder', onehotencoder, [1,2,3]),],
    remainder="passthrough").fit_transform(X)


In [12]:
#train-test split
trainingShare = 0.5 
seed  = 1
XTrain, XTest, yTrain, yTest=train_test_split(X, y, train_size=trainingShare,
                                              test_size = 1-trainingShare,
                                              random_state=seed)

# Input Scaling
sc = StandardScaler()
XTrain = sc.fit_transform(XTrain)
XTest = sc.transform(XTest)




In [ ]:
# One-hot's of the target vector
Y_train_onehot, Y_test_onehot = onehotencoder.fit_transform(yTrain), onehotencoder.fit_transform(yTest)

"""from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

lambdas=np.logspace(-5,7,13)
parameters = [{'C': 1./lambdas, "solver":["lbfgs"]}]#*len(parameters)}]
scoring = ['accuracy', 'roc_auc']
logReg = LogisticRegression()
gridSearch = GridSearchCV(logReg, parameters, cv=5, scoring=scoring, refit='roc_auc')"""